In [1]:
from simtk import openmm, unit
from simtk.openmm import app
import numpy as np

from copy import deepcopy
from tqdm import tqdm

import mdtraj
import nglview

In [2]:
def compute_potential(system, positions=None):
    """Print the potential energy given a System and positions."""
    integrator = openmm.VerletIntegrator(1.0 * unit.femtoseconds)
    context = openmm.Context(system, integrator)
    context.setPositions(positions)
    #print('Potential energy: %s' % context.getState(getEnergy=True).getPotentialEnergy())
    #context.getState(getEnergy=True).getPotentialEnergy()
    del context, integrator
    
def write_pdb(topology, positions, filename):
    with open(filename, 'w') as outfile:
        app.PDBFile.writeFile(topology, positions, outfile)
        
def make_view(topology, positions):
    mdtraj_aux_topology = mdtraj.Topology.from_openmm(topology)
    traj_aux = mdtraj.Trajectory(positions/unit.nanometers, mdtraj_aux_topology)
    view = nglview.show_mdtraj(traj_aux)
    view.clear()
    view.add_ball_and_stick('all')
    view.center()
    return view

In [3]:
pH=7.0
forcefield = app.ForceField('amber14-all.xml')

In [4]:
receptor_pdb       = app.PDBFile('testsystems/E9034A_ETEC/longus_E9034A_ETEC.pdb')

receptor_modeller  = app.Modeller(receptor_pdb.topology, receptor_pdb.positions)
addHs_receptor_out = receptor_modeller.addHydrogens(forcefield, pH=pH)

receptor_topology  = receptor_modeller.getTopology()
receptor_positions = receptor_modeller.getPositions()

write_pdb(receptor_topology,receptor_positions,'receptor_init.pdb')

In [5]:
view_receptor=make_view(receptor_topology,receptor_positions)
view_receptor

NGLWidget()

In [6]:
ligand_pdb      = app.PDBFile('testsystems/E9034A_ETEC/longus_E9034A_ETEC.pdb')

ligand_modeller = app.Modeller(ligand_pdb.topology, ligand_pdb.positions)
addHs_ligand_out= ligand_modeller.addHydrogens(forcefield, pH=pH)

ligand_topology  = ligand_modeller.getTopology()
ligand_positions = ligand_modeller.getPositions()

ligand_positions_0 = deepcopy(ligand_positions)

write_pdb(ligand_topology,ligand_positions,'ligand_init.pdb')

In [7]:
view_ligand=make_view(receptor_topology,receptor_positions)
view_ligand


NGLWidget()

In [8]:
offset_init=[10.0,0.0,0.0]*unit.nanometers

for ind_atom in range(len(ligand_positions)):
    ligand_positions[ind_atom]=ligand_positions[ind_atom]+offset_init

complex_modeller  = app.Modeller(receptor_topology, receptor_positions)
complex_modeller.add(ligand_topology, ligand_positions)

complex_topology  = complex_modeller.getTopology()
complex_positions = complex_modeller.getPositions()

write_pdb(complex_topology,complex_positions,'complex_init.pdb')

In [9]:
view_complex=make_view(complex_topology,complex_positions)
view_complex

NGLWidget()

In [10]:
system = forcefield.createSystem(complex_topology, nonbondedMethod=app.NoCutoff, 
                                 constraints=app.HBonds, implicitSolvent=app.OBC2)

In [11]:
compute_potential(system,complex_positions)

In [12]:
len(complex_positions)

5230

In [13]:
xx=np.linspace(0.2,10.0,200)

for new_x in tqdm(xx):
    offset_init=[new_x,0.0,0.0]*unit.nanometers
    for ind_atom in range(len(ligand_positions)):
        ligand_positions[ind_atom]=ligand_positions_0[ind_atom]+offset_init
    complex_modeller  = app.Modeller(receptor_topology, receptor_positions)
    complex_modeller.add(ligand_topology, ligand_positions)
    complex_topology  = complex_modeller.getTopology()
    complex_positions = complex_modeller.getPositions()
    compute_potential(system,complex_positions)

100%|██████████| 200/200 [02:26<00:00,  1.36it/s]
